In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pdfkit
import urllib.parse
import os
import concurrent.futures
import time

In [2]:
path_wkhtmltopdf = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'  # Thay đổi theo đường dẫn của bạn
config = pdfkit.configuration(wkhtmltopdf=path_wkhtmltopdf)
options = {
    'no-stop-slow-scripts': None,
}

In [17]:
df = pd.read_csv('cids_fail_round2.csv')

# Chuyển đổi cột đầu tiên thành danh sách
cids_list = df.iloc[:, 0].tolist()  # Lấy cột đầu tiên

# In ra danh sách
print(cids_list)

['NmJ5WHc3WSt4VVI2V0VEQ2UySlBPdz09', 'Nk9SMW9aZUlRYk05VElGWnVVeVBaQT09', 'TVRHVGZob1M3Rkluc1NtaDFucm1ndz09', 'MFhDdjFCV0N5SUZMd21QNmttbDlvZz09', 'aCtzQVM4eUtqNjVndWd1UElMaHhWdz09', 'cU01NHA5Sk5UN1g0RWFVQWcxV2hGUT09']


In [18]:
cids_list[0]

'NmJ5WHc3WSt4VVI2V0VEQ2UySlBPdz09'

In [5]:
def download_cv(cid, download_folder):
    base_url = "https://employer.jobsgo.vn/candidate/detail"
    params = {
        "cid": cid
    }
    cid_id = cid[-5:]
    # Headers
    headers = {
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Cookie": "colorgb-employer=ars0g0ndqor5g1ibudmnbed0gb; _identity-colorgb-employer=b09b08c1d2b4e46c3f112b5cd5f9b660b4ef707832b3c7061f29eac2cdd0c08ba%3A2%3A%7Bi%3A0%3Bs%3A26%3A%22_identity-colorgb-employer%22%3Bi%3A1%3Bs%3A22%3A%22%5B256330984%2C%22%22%2C2592000%5D%22%3B%7D; _csrf-colorgb-employer=6e54d3978a91607f4bb1a6ca36b2ac47cacd35461e09efc0a770d42cff03b1d9a%3A2%3A%7Bi%3A0%3Bs%3A22%3A%22_csrf-colorgb-employer%22%3Bi%3A1%3Bs%3A32%3A%22plo_dhvcuuM3upxK7yrhLN4mcpXkrcwh%22%3B%7D; last_logout=593d7738957e8373e82bcb944e31a3e0ab199588d6a42b5ca71a5616096b9a45a%3A2%3A%7Bi%3A0%3Bs%3A11%3A%22last_logout%22%3Bi%3A1%3Bi%3A1728444905%3B%7D; jobsgo-subemployer-logged-161233-256330984=1; alert-info=21d557c682fc76d6dfb3940f0540fd0ce90331f46673c55dc69792c561dbe1e5a%3A2%3A%7Bi%3A0%3Bs%3A10%3A%22alert-info%22%3Bi%3A1%3Bi%3A5%3B%7D",
        "DNT": "1",
        "Pragma": "no-cache",
        "Referer": "https://employer.jobsgo.vn/job/detail/19037635789?tab=suggest",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest",
        "sec-ch-ua": "\"Google Chrome\";v=\"129\", \"Not=A?Brand\";v=\"8\", \"Chromium\";v=\"129\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\""
    }
    print(f"{base_url}?cid={cid}")
    response = requests.get(base_url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Loại bỏ tất cả các thẻ <script>
    for script in soup.find_all('script'):
        script.decompose()

    # Loại bỏ tất cả các thẻ <div class="modal">
    for modal in soup.find_all('div', class_='modal'):
        modal.decompose()

    # Lưu thẻ <section class="portfolio" vào biến trước khi xóa
    portfolio_session = soup.find('section', class_='portfolio')

    # Kiểm tra xem thẻ có tồn tại không trước khi xóa
    if portfolio_session:
        # Lưu lại nội dung của portfolio_session nếu cần
        portfolio_session_copy = str(portfolio_session)
        portfolio_session_copy = BeautifulSoup(portfolio_session_copy)
        # Xóa thẻ trong soup
        portfolio_session.decompose()
    else:
        True
        portfolio_session_copy = BeautifulSoup("")

    name = soup.find('div', class_='profile').find('h1').get_text()

    filename_html_pdf = f"{name}_{cid_id}_html.pdf"
    html_content = str(soup)
    # Lưu pdf
    pdf_file_path = os.path.join(download_folder, 'test1.pdf')
    try:
        pdfkit.from_string(html_content, pdf_file_path, configuration=config, options=options)
        print("PDF đã được tạo thành công.")
    except Exception as e:
        print(f"Không thể tạo PDF, lỗi: {e}")

    try:
        os.rename(pdf_file_path, os.path.join(download_folder, filename_html_pdf))
        print(f"File đã được đổi tên thành: {filename_html_pdf}")
    except Exception as e:
        print(f"Không thể đổi tên file, lỗi: {e}")

    if portfolio_session_copy:
        # Tìm tất cả các iframe trong section
        links = portfolio_session_copy.find_all('iframe')
        
        # In ra danh sách các đường link và ngày cập nhật
        print("Danh sách các đường link và ngày cập nhật:")
        for i, link in enumerate(links):
            original_url = link.get('src')

            # Tách phần query từ URL
            parsed_url = urllib.parse.urlparse(original_url)
            query_params = urllib.parse.parse_qs(parsed_url.query)

            # Lấy giá trị của tham số 'file' và giải mã
            if 'file' in query_params:
                encoded_file_url = query_params['file'][0]
                decoded_file_url = urllib.parse.unquote(encoded_file_url)

                # In ra đường link đã được giải mã
                print("Đường link mới:", decoded_file_url)

                # Tìm tất cả các thẻ <p> trước thẻ iframe
                previous_paragraphs = link.find_all_previous('p')

                # Lọc các thẻ <p> chứa "Cập nhật ngày"
                update_paragraphs = [p for p in previous_paragraphs if 'Cập nhật ngày' in p.get_text()]

                # Lấy thẻ <p> đầu tiên (lớn nhất) nếu tồn tại
                if update_paragraphs:
                    latest_update_paragraph = update_paragraphs[0]  # Thẻ <p> đầu tiên trong danh sách
                    update_date = latest_update_paragraph.get_text().split(': ')[1]  # Lấy phần sau dấu ':'
                    update_date = update_date.replace("/","-")
                    print(f"Ngày cập nhật: {update_date}")
                else:
                    print("Không tìm thấy ngày cập nhật.")


            # URL của file PDF
            url = decoded_file_url

            # Gửi yêu cầu GET để tải file
            response = requests.get(url)

            # Kiểm tra xem yêu cầu có thành công không
            if response.status_code == 200:
                # Tên file để lưu
                
                filename = f"{name}_{update_date}_{cid_id}_download_{i+1}.pdf"
                pdf_file_download_path = os.path.join(download_folder, filename)
                # Ghi nội dung của response vào file
                with open(pdf_file_download_path, 'wb') as file:
                    file.write(response.content)
                
                print(f"File đã được tải xuống và lưu thành {pdf_file_download_path}.")
            else:
                print(f"Không thể tải file, mã lỗi: {response.status_code}")
    else:
        True


In [6]:

def chunk_list(data, chunk_size):
    """Chia danh sách thành các phần nhỏ."""
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

In [19]:
def handle_chunk(chunk, folder, index):
    """Xử lý từng chunk, theo dõi thời gian và lưu cid không thành công."""
    for cid in chunk:
        start_time = time.time()  # Ghi lại thời gian bắt đầu
        print(f"Đang tải CV {cid} trong chunk {index + 1}...")

        try:
            download_cv(cid, folder)  # Gọi hàm tải CV
            elapsed_time = time.time() - start_time  # Tính thời gian đã chạy

            # Kiểm tra xem có quá thời gian không
            if elapsed_time > 60:
                print(f"Thời gian quá lâu cho CV {cid}, sẽ lưu lại để thử lại sau.")
                failed_cids.append(cid)  # Lưu lại cid không thành công
        except Exception as e:
            print(f"Đã xảy ra lỗi khi tải CV {cid}: {e}")
            failed_cids.append(cid)  # Lưu lại cid không thành công

    # Nếu có cid không thành công, in ra danh sách
    if failed_cids:
        print(f"Các CV không tải được trong chunk {index + 1}: {failed_cids}")

In [16]:
# df = pd.DataFrame(failed_cids, columns=['cid'])

# # Ghi DataFrame vào file CSV
# df.to_csv('cids_fail_round2.csv', index=False)

In [20]:
len(cids_list)

6

In [21]:


# Số lượng luồng
num_threads = 10

# Chia danh sách cids_list thành các phần
cids_chunks = list(chunk_list(cids_list, len(cids_list) // num_threads + 1))

# Danh sách để lưu lại các cid không thành công
failed_cids = []

# Sử dụng ThreadPoolExecutor để thực hiện multithreading
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = []
    
    # Gửi các nhiệm vụ tải CV cho từng phần
    for index, chunk in enumerate(cids_chunks):
        download_folder = f'cv_downloaded/chunk_{index+1}'  # Tạo tên thư mục cho chunk
        os.makedirs(download_folder, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại
        
        # Tạo một nhiệm vụ cho từng chunk
        futures.append(executor.submit(handle_chunk, chunk, download_folder, index))

    # Chờ tất cả các nhiệm vụ hoàn thành
    for future in concurrent.futures.as_completed(futures):
        try:
            future.result()  # Nếu có lỗi, nó sẽ ném ra ngoại lệ
        except Exception as e:
            print(f"Đã xảy ra lỗi: {e}")

print("Tất cả các CV đã được tải.")




Đang tải CV NmJ5WHc3WSt4VVI2V0VEQ2UySlBPdz09 trong chunk 1...
https://employer.jobsgo.vn/candidate/detail?cid=NmJ5WHc3WSt4VVI2V0VEQ2UySlBPdz09
Đang tải CV Nk9SMW9aZUlRYk05VElGWnVVeVBaQT09 trong chunk 2...
https://employer.jobsgo.vn/candidate/detail?cid=Nk9SMW9aZUlRYk05VElGWnVVeVBaQT09
Đang tải CV TVRHVGZob1M3Rkluc1NtaDFucm1ndz09 trong chunk 3...
https://employer.jobsgo.vn/candidate/detail?cid=TVRHVGZob1M3Rkluc1NtaDFucm1ndz09
Đang tải CV MFhDdjFCV0N5SUZMd21QNmttbDlvZz09 trong chunk 4...
https://employer.jobsgo.vn/candidate/detail?cid=MFhDdjFCV0N5SUZMd21QNmttbDlvZz09
Đang tải CV aCtzQVM4eUtqNjVndWd1UElMaHhWdz09 trong chunk 5...
https://employer.jobsgo.vn/candidate/detail?cid=aCtzQVM4eUtqNjVndWd1UElMaHhWdz09
Đang tải CV cU01NHA5Sk5UN1g0RWFVQWcxV2hGUT09 trong chunk 6...
https://employer.jobsgo.vn/candidate/detail?cid=cU01NHA5Sk5UN1g0RWFVQWcxV2hGUT09
Không thể tạo PDF, lỗi: wkhtmltopdf reported an error:
Exit with code 1 due to network error: ProtocolUnknownError

File đã được đổi tên thành